In [ ]:
import tensorflow as tf
import scipy
import sklearn
import numpy as np
import sys
#from keras_tcn import TCN
#import tensorflow_addons as tfa
import typeguard

print(f"TensorFlow version: {tf.__version__}")
print(f"SciPy version: {scipy.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"numpy version: {np.__version__}")
print(f"Python version: {sys.version}")

只預測右腳

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import register_keras_serializable  # 改用這個路徑

import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, TimeDistributed 
from sklearn import preprocessing
import math
import socket
from sklearn.preprocessing import MinMaxScaler
import threading
import queue
from collections import deque
import time
import asyncio
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
import nest_asyncio
import joblib
from datetime import datetime
import numba
import logging
import psutil

# 更專業的日誌配置
def setup_logging():
    """設置日誌記錄系統"""
    logging.basicConfig(
        level=logging.DEBUG,
        format='%(asctime)s.%(msecs)03d | %(levelname)-8s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
            logging.FileHandler('/home/ndt/Desktop/predict_model/comprehensive_log.log'),
            logging.StreamHandler()  # 同時輸出到控制台
        ]
    )
    return logging.getLogger(__name__)

#預測函數
def predict_data10step_graph(model_predict, data_min, data_max, feature_min, feature_max):
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 7], dtype=tf.float32)])
    def predict_fn(imu_data_40step):
        start_time = tf.timestamp()
        
        past_data = imu_data_40step[-40:, 5:6]
        past_data_scaled = (past_data - data_min) / (data_max - data_min + 1e-8)
        past_data_scaled = past_data_scaled * (feature_max - feature_min) + feature_min
        past_data_scaled = tf.reshape(past_data_scaled, [1, 40, 1])
        
        decoder_input_test = tf.zeros([1, 10, 1], dtype=tf.float32)
        predictions_ta = tf.TensorArray(dtype=tf.float32, size=10)
        
        def loop_body(step, decoder_input_test, predictions_ta):
            prediction_step = model_predict([past_data_scaled, decoder_input_test])
            new_prediction = prediction_step[0, 0, 0]
            predictions_ta = predictions_ta.write(step, new_prediction)
            if step < 9:
                decoder_input_test = tf.tensor_scatter_nd_update(
                    decoder_input_test, [[0, step + 1, 0]], [new_prediction]
                )
            return step + 1, decoder_input_test, predictions_ta
        
        # Use tf.while_loop with all state variables
        step = tf.constant(0)
        step, decoder_input_test, predictions_ta = tf.while_loop(
            lambda step, _, __: step < 10,  # Condition only checks step
            loop_body,
            [step, decoder_input_test, predictions_ta]
        )
        
        predictions_scaled = predictions_ta.stack()
        predictions_orig = predictions_scaled * (data_max - data_min) / (feature_max - feature_min) + data_min
        
        future_data = tf.zeros([10, 7], dtype=tf.float32)
        indices = tf.stack([tf.range(10), tf.fill([10], 5)], axis=1)
        future_data = tf.tensor_scatter_nd_update(future_data, indices, predictions_orig)
        
        full_data = tf.concat([imu_data_40step[-40:, :], future_data], axis=0)
        
        end_time = tf.timestamp()
        exec_time = end_time - start_time
        #tf.print(f"predict_data10step execution time: {exec_time} seconds")
        return full_data
    return predict_fn

def detectHS_graph(model_HS):
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 7], dtype=tf.float32)])
    def detect_fn(imu_data_50step):
        start_time = tf.timestamp()
        # processed_data：直接使用 imu_data_50step（假设数据已完整）
        processed_data = imu_data_50step[-50:, 1:7]

        # 使用 TensorFlow 的標準化，模仿 sklearn 的行为
        mean, variance = tf.nn.moments(processed_data, axes=[0])
        input_scaled = (processed_data - mean) / tf.sqrt(variance + 1e-8)
        input_scaled = tf.reshape(input_scaled, [1, 50, 6])
        
        # 使用GPU执行HS模型预测，直接调用模型
        hs_level = model_HS(input_scaled)
        
        # 使用 tf.cond 替代 if 语句
        cond1 = hs_level[0, 49, 0] > 0.015
        cond2 = imu_data_50step[49, 5] * 180 / 3.14 < -10
        signal = tf.cond(
            tf.logical_and(cond1, cond2), 
            lambda: tf.constant(1, dtype=tf.int32), 
            lambda: tf.constant(0, dtype=tf.int32)
        )
        
        end_time = tf.timestamp()
        exec_time = end_time - start_time
        #tf.print(f"{datetime.now()} detectHS execution time:", exec_time, "seconds")
        return signal
    return detect_fn

@tf.function(input_signature=[
    tf.TensorSpec(shape=[10000, 7], dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int32),
    tf.TensorSpec(shape=(), dtype=tf.int32)
])
def combined_predict_and_detect(data_5610_tensor, data_count, data_index):
    total_start = tf.timestamp()

    # 始終使用 data_index 提取最近 40 筆數據
    if data_index == 0:
        current_data = data_5610_tensor[-40:, :]
    else:
        current_data = tf.concat(
            [data_5610_tensor[data_index:], data_5610_tensor[:data_index]],
            axis=0
        )[-40:, :]

    if data_count < 10000:
        current_data = data_5610_tensor[:data_count][-40:, :]
    else:
        current_data = tf.concat(
            [data_5610_tensor[data_index:], data_5610_tensor[:data_index]],
            axis=0
        )[-40:, :]
        
    start_time = tf.timestamp()
    data_5610_50step = predict_data10step_R(current_data)
    end_time = tf.timestamp()
    tf.print("predict_data10step_R execution time:", end_time - start_time, "seconds")

    start_time = tf.timestamp()
    signal = detectHS(data_5610_50step)
    end_time = tf.timestamp()
    tf.print("detectHS execution time:", end_time - start_time, "seconds")

    total_end = tf.timestamp()
    tf.print("Total combined_predict_and_detect execution time:", total_end - total_start, "seconds")
    return signal

# 重建原本detectHS的模型，因為這邊環境跟ndt_cpu不一樣
model_HS = tf.keras.Sequential([
    tf.keras.layers.LSTM(
        units=64,
        return_sequences=True,  # 必須設為 True 來保持序列長度
        input_shape=(50, 6),    # 從原始配置中的 batch_input_shape
        dropout=0.2,
        recurrent_dropout=0.1,
        name='lstm_2'           # 使用相同的層名稱
    ),
    tf.keras.layers.Dense(
        units=1,
        name='dense_2'
    ),
    tf.keras.layers.Activation(
        'sigmoid',
        name='activation_2'
    )
])

# 編譯模型
model_HS.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 打印模型摘要以驗證結構
#model_HS.summary()

# 嘗試載入權重
try:
    model_HS.load_weights("/home/ndt/Desktop/predict_model/model_weights.h5")
    detectHS = detectHS_graph(model_HS)
    print(f"{datetime.now()} 權重載入成功！")
except Exception as e:
    print("載入權重時出錯：", e)

#以下是預測HS模型建置

# 1. 首先定義相同的 PositionalEncoding 層
@register_keras_serializable() # 使用 register_keras_serializable 直接註冊
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, maxlen, embed_dim, **kwargs):  # 添加 **kwargs 來接受額外的參數
        super(PositionalEncoding, self).__init__(**kwargs)  # 將 **kwargs 傳遞給父類
        self.pos_emb = self.add_weight(name="pos_emb",
                                     shape=(1, maxlen, embed_dim))
        # 保存參數供序列化使用
        self.maxlen = maxlen
        self.embed_dim = embed_dim

    def call(self, x):
        return x + self.pos_emb

    # 加入這個方法以支援序列化
    def get_config(self):
        config = super().get_config()
        config.update({
            "maxlen": self.maxlen,
            "embed_dim": self.embed_dim
        })
        return config
custom_objects = {"PositionalEncoding": PositionalEncoding}
#model_predictL = keras.models.load_model('/mnt/c/Users/m2023/Desktop/vision/predict_model/modelSeq2Seq LSTM with Positional Encoding and Multi-Head Attention For 1 Input For Left20250225.keras', custom_objects=custom_objects) #predict IMU for left
model_predictR = keras.models.load_model('/home/ndt/Desktop/predict_model/modelSeq2Seq LSTM with Positional Encoding and Multi-Head Attention For 1 Input For Right20250225.keras', custom_objects=custom_objects) #predict IMU for right 
scaler_right = joblib.load('/home/ndt/Desktop/predict_model/right_scaler_20250225.pkl')
scaler_left = joblib.load('/home/ndt/Desktop/predict_model/left_scaler_20250225.pkl')

# 提取 scaler 參數
data_min_right = float(scaler_right.data_min_[0])
data_max_right = float(scaler_right.data_max_[0])
feature_min_right = scaler_right.feature_range[0]
feature_max_right = scaler_right.feature_range[1]


#predict_data10step_L = predict_data10step_graph(model_predictL, scaler_left)
# 定義預測函數
predict_data10step_R = predict_data10step_graph(
    model_predictR,
    data_min_right,
    data_max_right,
    feature_min_right,
    feature_max_right
)


#model_predictL.summary()


# 初始化狀態機狀態
state = "IDLE"  # 初始狀態
step_counter = 0  # 用於記錄新增的步數
start_index = 0
predict_state = 1
running = True  # 設置一個標誌來控制程式運行
signal = 0


cue_step = 5 #提前幾步cue
TRIAL_NAME = '20250306-1HS'
output_file = "/mnt/c/Users/m2023/Desktop/experience_data/"

class DataProcessor:
    def __init__(self, logger):
        self.logger = logger
        self.data_queue = queue.Queue()
        self.running_event = threading.Event()
        self.running_event.set()
        self.prediction_thread = None  # 新增預測執行緒控制
        self.prediction_active = threading.Event()  # 控制預測執行緒的運行
        
        # 預測相關的變量
        self.predict_state = None
        self.state = "IDLE"
        #self.data_5610 = np.array([[1,2,3,4,5,6,7]])
        #self.data_5611 = np.array([[1,2,3,4,5,6,7]])
        with tf.device('/GPU:0'):
            self.data_5610_tensor = tf.Variable(np.zeros((10000, 7), dtype=np.float32))  # 初始為 1000x7 的零陣列
            #self.latest_40_data = tf.Variable(np.zeros((40, 7), dtype=np.float32))
        self.data_index = 0  # 下一個數據插入的位置
        self.data_count = 0  # 已接收的數據總數，最大 1000

        self.data_lock = threading.Lock()  # 只用於寫入保護

        self.R_HeelStrike_Predict_Time=np.array([[1]])
        self.L_HeelStrike_Predict_Time=np.array([[1]])

        self.execution_overtime_count = 0


        
        
        # 創建進程池用於預測
        #self.process_pool = ProcessPoolExecutor(max_workers=1)

        # 修改進程池的創建方式，改為需要時才創建
        self.process_pool = None

    def _init_process_pool(self):
        """初始化或重新初始化進程池"""
        if hasattr(self, 'process_pool'):
            self.process_pool.shutdown(wait=False)
        self.process_pool = ProcessPoolExecutor(max_workers=1)
    
    def receive_data(self, conn):
        """接收數據的線程函數"""
        buffer = ""
        while self.running_event.is_set():
            try:
                data = conn.recv(1024)  # 增加緩衝區大小
                if not data:
                    print("客戶端已斷開連接")
                    self.running_event.clear()
                    break
                # 將接收到的數據追加到緩衝區
                buffer += data.decode()
                # 分割完整的消息（考慮多個換行符的情況）
                messages = buffer.split('\n')
                # 最後一個元素可能是不完整的消息，使用messages[-1]讀最後一個元素保留在buffer中
                buffer = messages[-1]

                # messages[:-1]是指除了除了最後一個元素以外的所有完整消息
                for message in messages[:-1]: 
                    if message.strip():  # 確保消息不是空字符串
                        self.data_queue.put(message.strip())
                        #print(f"已放入隊列：{message}")

            except Exception as e:
                print(f"接收數據時發生錯誤：{e}")
                self.running_event.clear()
                break

    def process_data(self):
        """處理數據的線程函數"""
        batch_size = 1  # 批量處理大小
        batch = []
        
        while self.running_event.is_set():
            try:
                # 批量收集數據
                while len(batch) < batch_size:
                    try:
                        message = self.data_queue.get()
                        batch.append(message)
                        if self.data_queue.qsize() > 100:
                            self.logger.info(f"警告：佇列堆積，當前大小: {self.data_queue.qsize()}")
                    except queue.Empty:
                        break

                if not batch:
                    #time.sleep(0.001)  # 短暫休眠避免CPU空轉
                    continue

                # 批量處理數據
                for message in batch:
                    self._process_single_message(message)
                
                batch = []  # 清空批次

            except Exception as e:
                print(f"處理數據時發生錯誤：{e}")
                continue

    def _process_single_message(self, message):
        """處理單條消息"""
        try:
            if message.startswith("data_5610:"):
                self._handle_data_5610(message)
                self.logger.info(message)

            elif message == "ready to predict right leg":
                self.logger.info("ready to predict right leg")
                self.predict_state = 1
                self.state = "IDLE"
                self.start_prediction_thread()  # 啟動持續預測thread
            elif message == "ready to predict left leg":
                self.logger.info("ready to predict left leg")
                self.predict_state = 0
                self.state = "IDLE"
                self.prediction_active.clear()  # 停止預測

            elif message == "END":
                self._handle_end()

        except Exception as e:
            self.logger.info(f"處理消息時發生錯誤：{e}")

    def _handle_data_5610(self, message):
        """處理5610數據"""
        start_time = time.time()  # 記錄開始時間
        latest_data_5610 = message.replace("data_5610:", "").strip()
        latest_data_5610 = np.array(list(map(float, latest_data_5610.split(",")))).reshape(1, -1)
        with tf.device('/GPU:0'):
            latest_data_5610 = tf.convert_to_tensor(latest_data_5610, dtype=tf.float32) #
        with self.data_lock:  # 只在寫入時鎖定
            #self.data_5610 = np.vstack([self.data_5610 , latest_data_5610])
            self.data_5610_tensor[self.data_index].assign(latest_data_5610[0])
            # 更新索引，實現循環緩衝
            self.data_index = (self.data_index + 1) % 10000
            # 更新計數，最大為 10000
            if self.data_count < 10000:
                self.data_count += 1
            self.logger.info(f"Updated data_count: {self.data_count}")  # 添加日誌
    
        #print("處理5610數據")
        end_time = time.time()  # 記錄結束時間
        execution_time = end_time - start_time  # 計算執行時間
        #print(f"_handle_data_5610執行時間: {execution_time:.6f} 秒")  # 輸出執行時間
        self.logger.info(f"_handle_data_5610執行時間: {execution_time:.6f} 秒")

    def start_prediction_thread(self):
        """啟動持續預測執行緒"""
    # 如果已有執行中的預測執行緒，先停止它
        if self.prediction_thread and self.prediction_thread.is_alive():
            self.prediction_active.clear()
            self.prediction_thread.join()
        
        # 重置並啟動新的預測執行緒
        self.prediction_active.set()
        self.prediction_thread = threading.Thread(target=self.prediction)
        self.prediction_thread.start()

    def prediction(self):
        """持續執行預測的執行緒函數"""
        while self.prediction_active.is_set() and self.running_event.is_set():
            try:
                if self.predict_state == 1 and self.data_count > 40:
                    self.logger.info("持續預測右腳")
                    result = self._handle_right_prediction()
                    if not result:  # 若返回 False，退出
                        break
            except Exception as e:
                self.logger.info(f"持續預測過程發生錯誤：{e}")

    #@numba.jit(nopython=True)  # 使用 Numba 編譯關鍵邏輯    
    def _handle_right_prediction(self):
        """處理右腳預測"""
        self.logger.info("處理右腳預測")
        if self.state == "IDLE":
            try:
                if self.state == "IDLE" and self.data_count > 40:
                    start_time = time.time()
                    self.start_index = self.data_index

                    mid1 = time.time()
                    self.logger.info(f"1執行時間: {mid1 - start_time:.6f} 秒")

                    profiler_options = tf.profiler.experimental.ProfilerOptions(
                        host_tracer_level=3,
                        device_tracer_level=2,
                        python_tracer_level=1
                    )
                    #tf.profiler.experimental.start('/mnt/c/Users/m2023/AppData/Local/Programs/Microsoft VS Code/logdir')
                    with tf.device('/GPU:0'):
                        future = combined_predict_and_detect(self.data_5610_tensor, self.data_count, self.data_index)
                    #tf.profiler.experimental.stop()

                    mid2 = time.time()
                    self.logger.info(f"Call time: {mid2 - mid1 :.6f} 秒")  # 不含同步

                    sync_start = time.time()
                    signal = future.numpy()  # 顯式同步
                    sync_end = time.time()
                    self.logger.info(f"Sync time: {sync_end - sync_start :.6f} 秒")

                    mid3 = time.time()
                    self.logger.info(f"3執行時間: {mid3 - mid1 :.6f} 秒")

                    self.logger.info(f"signal右腳 {signal}")
                    end_time = time.time()
                    execution_time = end_time - start_time
                    self.logger.info(f"_handle_right_prediction執行時間: {execution_time:.6f} 秒")

                if signal == 1:
                    self.logger.info("右腳未來10步有 HeelStrike")
                    self.state = "MONITOR_Right"
                
                # if execution_time > 0.05:
                #     self.logger.warning("預測時間超過0.045秒，可能跟不上數據速率")
                #     self.execution_overtime_count = getattr(self, 'execution_overtime_count', 0) + 1

            except Exception as e:
                self.logger.error(f"{datetime.now()} 右腳預測發生錯誤：{e}", exc_info=True)

        elif self.state == "MONITOR_Right":
            self.logger.info("開始右腳記步")
            self._monitor_right_steps()
            if not self._monitor_right_steps():  # 檢查計步返回值
                return False
        
        return True  # 繼續預測


    def _monitor_right_steps(self):
        """監控右腳步數"""

        last_printed_step = -1  # 記錄上一次打印的步數
        while self.prediction_active.is_set() and self.running_event.is_set():
            
            #current_length = len(self.data_5610)
            with self.data_lock:
                current_index = self.data_index  # 獲取當前的 data_index
            new_steps = (current_index - self.start_index) % 10000  # 使用 data_index 計算新步數
            
            if new_steps != last_printed_step:  # 僅在步數變化時打印
                last_printed_step = new_steps
                self.logger.info(f"右腳步數第 {new_steps}")
                self.logger.info(f"current_index: {current_index}, start_index: {self.start_index}, new_steps: {new_steps}")
            if new_steps == cue_step:
                self.logger.info("亮右邊")
                #self._send_signal("0,1")

            if new_steps == 10:
                with self.data_lock:  # 只在寫入結果時鎖定
                    latest_index = (self.data_index - 1) % 10000
                    predicted_time = self.data_5610_tensor[latest_index, 0]
                    self.logger.info(f"Writing predicted time {predicted_time} at index {latest_index}")  # 调试日志
                    self.R_HeelStrike_Predict_Time = np.vstack([
                        self.R_HeelStrike_Predict_Time, 
                        self.data_5610_tensor[latest_index, 0]
                    ])
                self.logger.info("右腳10步")
                self.start_index = 0
                self.state = "IDLE"
                self.prediction_active.clear()  # 停止預測線程
                return False  # 返回False表示要停止預測
                
            if new_steps > 10:            
                self.start_index = 0
                self.state = "IDLE"
                self.prediction_active.clear()  # 停止預測線程
                return False  # 返回False表示要停止預測
            #time.sleep(0.005)

#建立TCP For Unity,用於send
    def TCP_For_Unity(self, host, port):
        try:
            # 定义Unity服务器的IP地址和端口号
            unity_server_address = (host, port)  # 这里的IP地址和端口号应该与Unity服务器的设置相匹配

            # 创建一个TCP/IP socket
            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

            # 连接到Unity服务器
            client_socket.connect(unity_server_address)

            # 发送信号到Unity服务器
            def send_signal(signal):
                message = signal.encode()  # 将信号转换为字节串
                client_socket.sendall(message)  # 发送信号
            return client_socket, send_signal
        except Exception as e:
            self.logger.error(f"TCP For Unity發送錯誤：{e}", exc_info=True)
    
#紀錄預測HS的真實時間
    def save_data(self):
        index_min=min(   len(self.R_HeelStrike_Predict_Time),  len(self.L_HeelStrike_Predict_Time)   )
        index_0=np.array([["rightfoot"]])
        ee = np.vstack([index_0,self.R_HeelStrike_Predict_Time[1:index_min]])
        index_0=np.array([["leftfoot"]])
        ff = np.vstack([index_0,self.L_HeelStrike_Predict_Time[1:index_min]])
        gg = np.hstack([ee,ff])
        gg_csv = pd.DataFrame(gg)
        gg_csv.to_csv(output_file + TRIAL_NAME + 'predicthstime.csv',header=0,index=0)
        self.logger.info(f"預測結果已成功保存到 {output_file}")


    def _handle_end(self):
        self.logger.info("結束信號接收，保存數據")
        self.prediction_active.clear()  # 停止預測執行緒
        if self.prediction_thread:
            self.prediction_thread.join()
        self.save_data()
        self.running_event.clear()  # 停止所有進程

#建立TCP For IMU接收,用於receive
    def TCP_For_IMU(self, host, port):
        try:
            server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            server.bind((host, port))
            server.listen(1)  # 允許 1 個連線
            self.logger.info("伺服器已啟動，等待連線...據")

            conn, addr = server.accept()
            self.logger.info(f"已連接來自 {addr} 的客戶端")
            return conn  # 回傳 conn 以供後續使用
        except Exception as e:
            self.logger.error(f"TCP For IMU接收錯誤：{e}", exc_info=True)   
          

    def start(self):
        """啟動服務器"""
        try:

            dummy_data = tf.random.normal((10000, 7), dtype=tf.float32)
            dummy_count = tf.constant(40, dtype=tf.int32)              # 假設有 40 筆資料
            dummy_index = tf.constant(0, dtype=tf.int32)               # 索引從 0 開始
            combined_predict_and_detect(dummy_data, dummy_count, dummy_index)
            self.logger.info("模型已預熱")

            # 1. 初始化 Unity 客戶端
            #client_socket, send_signal = self.TCP_For_Unity(host='localhost', port=12345)
            # 2. 啟動 IMU 伺服器
            conn = self.TCP_For_IMU(host='localhost', port=34567)
            setup_logging()  # 在程式啟動時呼叫

            
            
            # 創建並啟動線程
            receiver_thread = threading.Thread(
                target=self.receive_data, 
                args=(conn,)
            )
            processor_thread = threading.Thread(
                target=self.process_data
            )

            receiver_thread.start()
            processor_thread.start()


            # 等待線程結束
            receiver_thread.join()
            processor_thread.join()

            while self.running_event.is_set():
                process = psutil.Process()
                self.logger.info(f"Memory usage: {process.memory_info().rss / (1024 * 1024):.2f} MB")
                time.sleep(10)

        except Exception as e:
            self.logger.error(f"服務器錯誤：{e}", exc_info=True)
        finally:
            self.cleanup()

    def cleanup(self):
        """清理資源"""
        self.running_event.clear()
        if self.process_pool:
            self.process_pool.shutdown()

# 使用方式
if __name__ == "__main__":
    logger = setup_logging()
    processor = DataProcessor(logger)
    processor.start()  # 使用默認的 localhost:23456

KeyboardInterrupt: 